In [ ]:
# asume que los datos estan en ../data/<año>.csv.bz2
# fixme: poner el directorio de data que esté en el repo cuando pusheen los pibes

import seaborn as sns
import matplotlib.pyplot as plt

import fnmatch

import pandas as pd
import time
import os
import shutil
#sns.set(color_codes=True)

# load data
tmp_dir = "./tmp_csv/"
years = [str(year) for year in range(1994,1999)]

## Procesar datos

In [ ]:
#crear directorio con data de delays agrupada por retraso 
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
for year in years:
    if not os.path.exists(tmp_dir+year+".csv"):
        print("Leyendo csv: {}".format(year))
        t = time.time()
        df = pd.read_csv("../data/"+year+".csv.bz2", compression="bz2", usecols=["Month", "Year", "DayofMonth", "DayOfWeek", "DepDelay", "ArrDelay"], encoding="ISO 8859-1")    

        #formato mas feliz para fechas
        dates = pd.to_datetime(df.Year*10000+df.Month*100+df.DayofMonth,format='%Y%m%d')
        df["date"] = dates
        df = df[["date", "Year", "DepDelay", "ArrDelay"]]

        df = df.groupby(by=df['date'].dt.date).mean()
        df.to_csv(tmp_dir+year+".csv")
        del df 
    

In [ ]:
for filename in os.listdir(tmp_dir):
    if filename.endswith(".csv") and filename != "merged.csv": 
        print(filename)
        df = pd.read_csv(tmp_dir+filename)
        fig, ax = plt.subplots(figsize=(16,4))
        sns.lineplot(x=df.date, y=df.DepDelay, linestyle='-')
        plt.xticks(plt.xticks()[0], df.date, rotation=90)
        plt.tight_layout()
        xticks = ax.xaxis.get_major_ticks()
        for i in range(len(xticks)):
            if i%30 != 0:
                xticks[i].set_visible(False)
            if i%int((len(xticks))/4) == 0:    
                plt.axvline(x=i, color="orange")
            if i%int((len(xticks))/3) == 0:    
                plt.axvline(x=i, color="green")
        fig.autofmt_xdate()
        plt.show()
        
#Trimestres en verde, cuatrimestres en naranja

In [ ]:
#merge
df = pd.DataFrame(columns=["date", "DepDelay", "ArrDelay"])
for filename in sorted(os.listdir(tmp_dir)):
    if filename.endswith(".csv") and filename != "merged.csv":
        print(filename)
        df_input = pd.read_csv(tmp_dir+filename)
        df = pd.concat([df, df_input], sort=True)
        
df.set_index('date')
df.to_csv(tmp_dir+"merged.csv")

In [ ]:
df = pd.read_csv(tmp_dir+"merged.csv")
df.plot(x="date", y="DepDelay", linestyle='-', figsize=(16,4))

## CML acá 

In [ ]:
#delays = df["DepDelay"]
old_delays = df.loc[df["Year"]<1998]["DepDelay"]
print(old_delays)
print(df["DepDelay"])

In [ ]:
#para jugar con fases
time        = np.arange(0, 12,0.1);
amplitude   = np.sin(2 * math.pi / 1* time + math.pi)
plt.plot(time, amplitude)
plt.show()

In [ ]:
import numpy as np
import math

max_grade = 1
phases = [i*math.pi/2 for i in range(4)]

# semanas, meses, bimestres, trimestres, cuatrimestres, semestres, años
freqs =  [7] + [355/i for i in [12, 6, 4, 3, 2, 1]]

def trig_vals(t):
    return  [math.sin(2 * math.pi / f * t + p) for f in freqs for p in phases]

def get_x_vals(x):
    x_vals = []
    for i in range(len(x)):
        row = np.array([i**p for p in range(max_grade+1)] + trig_vals(i), dtype='float')
        x_vals.append(row)
    return x_vals

arrays = get_x_vals(old_delays)
A = np.stack(arrays)
print(A.shape)

#coefs = np.linalg.lstsq(A, old_delays, rcond=None)[0]
coefs = np.linalg.solve(A.T@A, A.T@old_delays)

In [ ]:
del df["dl_pred"]

In [ ]:
full_delays = df["DepDelay"]
full_arrays = get_x_vals(full_delays)
full_A = np.stack(full_arrays)

df["dl_pred"] = full_A@coefs
df

In [ ]:
plt.figure(figsize=(16,5))
plt.xlabel('Predicción para el i-ésimo dia')


ax1 = df.dl_pred.plot(color='orange', grid=True, label='pred delay')
ax2 = df.DepDelay.plot(color='blue', grid=True, secondary_y=True, label='real delay')

xticks = ax1.xaxis.get_major_ticks()
plt.axvline(x=len(old_delays), color="green")

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1+h2, l1+l2, loc=2)
plt.show()

## BORRAR ARCHIVOS

In [ ]:
#borrar aux de años
for filename in os.listdir(tmp_dir):
    if fnmatch.fnmatch(filename, '*.csv') and filename != "merged.csv":
        os.remove(tmp_dir+filename)

In [ ]:
#borrar merged
os.remove(tmp_dir+"merged.csv")

In [ ]:
#borrar tmp
shutil.rmtree(tmp_dir)